In [36]:
from dotenv import load_dotenv
import os
from espn_api.football import League

In [37]:
load_dotenv()

league_id = os.getenv('LEAGUE_ID')
swid = os.getenv('SWID')
espn_s2 = os.getenv('ESPN_S2')
year = 2024
week = 10
league_median_name = 'League Median'
against_league_median_team_id = 1

In [38]:
league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)

In [ ]:
def get_current_and_projected_scores_for_week(league, week, league_median_name):
    box_scores = league.box_scores(week = week)
    teams_and_current_scores = []
    teams_and_projected_scores = []
    for box_score in box_scores:
        team_one = box_score.home_team
        team_two = box_score.away_team
        team_one_score = box_score.home_score
        team_two_score = box_score.away_score
        team_one_projected_score = box_score.home_projected
        team_two_projected_score = box_score.away_projected

        if team_one.team_name != league_median_name:
            teams_and_current_scores.append((team_one.team_id, team_one_score))
            teams_and_projected_scores.append((team_one.team_id, team_one_projected_score))
        
        if team_two.team_name != league_median_name:
            teams_and_current_scores.append((team_two.team_id, team_two_score))
            teams_and_projected_scores.append((team_two.team_id, team_two_projected_score))

    return teams_and_current_scores, teams_and_projected_scores

def get_league_median_not_including_league_median_opponent(league, week, league_median_name, against_league_median_id):
    
    teams_and_current_scores, teams_and_projected_scores = get_current_and_projected_scores_for_week(league, week, league_median_name)

    sorted_teams_and_current_scores = sorted(teams_and_current_scores, key = lambda score: score[1])
    sorted_teams_and_projected_scores = sorted(teams_and_projected_scores, key = lambda score: score[1])

    filtered_sorted_teams_and_current_scores = [t for t in sorted_teams_and_current_scores if t[0] != against_league_median_id]
    filtered_sorted_teams_and_projected_scores = [t for t in sorted_teams_and_projected_scores if t[0] != against_league_median_id]

    below_league_median_index = -1
    above_league_median_index = -1
    n = len(filtered_sorted_teams_and_current_scores)
    if n % 2 == 1:
        below_league_median_index = n // 2
        above_league_median_index = n // 2
    else:
        below_league_median_index = n // 2 - 1
        above_league_median_index = n // 2

    current_median = (filtered_sorted_teams_and_current_scores[below_league_median_index][1] + filtered_sorted_teams_and_current_scores[above_league_median_index][1])/2
    projected_median = (filtered_sorted_teams_and_projected_scores[below_league_median_index][1] + filtered_sorted_teams_and_projected_scores[above_league_median_index][1])/2

    below_league_median_team_id = filtered_sorted_teams_and_current_scores[below_league_median_index][0]
    above_league_median_team_id = filtered_sorted_teams_and_current_scores[above_league_median_index][0]

    return current_median, projected_median, below_league_median_team_id, above_league_median_team_id

def get_current_data(league, current_week, league_median_name):
    teams_and_current_scores, teams_and_projected_scores = get_current_and_projected_scores_for_week(league, current_week, league_median_name)
    current_week_scores = {}

    for current_score_info, projected_score_info in zip(teams_and_current_scores, teams_and_projected_scores):
        team_id = current_score_info[0]
        current_score = current_score_info[1]
        projected_score = projected_score_info[1]

        info = {}
        info["currentScore"] = current_score
        info["projectedScore"] = projected_score
        current_week_scores[team_id] = info
    return current_week_scores

In [40]:
teams_and_current_scores, teams_and_projected_scores = get_current_and_projected_scores_for_week(league=league, week=week, league_median_name=league_median_name)
current_median, projected_median, below_league_median_team_id, above_league_median_team_id = get_league_median_not_including_league_median_opponent(league, week, league_median_name, against_league_median_team_id)

In [41]:
print(teams_and_current_scores)
print(teams_and_projected_scores)
print(current_median, projected_median, below_league_median_team_id, above_league_median_team_id)

[(2, 95.64), (8, 116.74), (9, 98.3), (1, 129.3), (3, 128.84), (7, 106.52), (5, 88.48), (4, 83.94), (6, 103.78)]
[(2, 121.33), (8, 112.12), (9, 120.84000000000002), (1, 122.61), (3, 114.43), (7, 123.60000000000001), (5, 112.5), (4, 113.03000000000002), (6, 111.72)]
101.03999999999999 113.73000000000002 9 6


In [42]:
current_week_scores = get_current_data(league, week, league_median_name)
current_week_scores

{2: {'currentScore': 95.64, 'projectedScore': 121.33},
 8: {'currentScore': 116.74, 'projectedScore': 112.12},
 9: {'currentScore': 98.3, 'projectedScore': 120.84000000000002},
 1: {'currentScore': 129.3, 'projectedScore': 122.61},
 3: {'currentScore': 128.84, 'projectedScore': 114.43},
 7: {'currentScore': 106.52, 'projectedScore': 123.60000000000001},
 5: {'currentScore': 88.48, 'projectedScore': 112.5},
 4: {'currentScore': 83.94, 'projectedScore': 113.03000000000002},
 6: {'currentScore': 103.78, 'projectedScore': 111.72}}